# Long Sentence Resolution - Evaluate Splitting Model

# Documentation

## Connect Google Drive

*   Run the cell "Connect Google Drive" and on the popup window click  "Connect to Google Drive".

*   In the new page opened, choose an account (your account) and click Allow.

## Configuration parameters

* **Source of test set generated predictions**: The source of test set generated predictions.

* **Source of test set ground-truth**: The source of test set ground-truth.

* **Target maximum #words**: The maximum number of words a sentence should have to be considered a valid target sentence (default: 16).

# Installations

In [ ]:
!pip install evaluate==0.3.0    \
             bert-score==0.3.11 \
             sacrebleu==2.3.1   \
             spacy==3.4.1       \
             tqdm==4.64.1

# Imports

In [ ]:
import os, evaluate

import numpy as np

import ipywidgets as widgets

import pandas as pd

from pathlib import Path

from typing import List, Union

from google.colab import drive

from spacy.lang.en import English

from spacy.tokens import Doc, Token, Span

from tqdm import tqdm

# Functions / Classes

In [ ]:
class DocumentAnalysis:
  def __init__ (self):
    self._linguistics_model = English()

    self._linguistics_model.add_pipe("sentencizer")

  def extract_sentence_n_words_from_documents(self, documents: Union[List[str], np.ndarray, pd.Series]) -> pd.Series:
    series_index = None if type(documents) != pd.Series else documents.index

    return pd.Series(list(map(self._extract_sentence_n_words_from_document, self._linguistics_model.pipe(documents, batch_size=50))), index=series_index)

  def _extract_sentence_n_words_from_document(self, doc: Doc) -> List[int]:
    return list(map(self._extract_n_words_from_span, doc.sents))

  def _extract_n_words_from_span(self, span: Span) -> int:
    return sum(map(self._token_is_word, span))

  def _token_is_word(self, token: Token) -> bool:
    return not any([ token.is_bracket,
                     token.is_punct,
                     token.is_currency,
                     token.is_quote,
                     token.is_space ]) # https://spacy.io/api/token#attributes

class SplittingWordBasedAccuracy:
  def __init__ (self, n_words_threshold: int):
    self._document_analysis = DocumentAnalysis()

    self._n_words_threshold = n_words_threshold

  def compute(self, documents: Union[List[str], np.ndarray, pd.Series]) -> float:
    if len(documents) == 0: return 0

    sample_level_accuracies = np.array(self._document_analysis.extract_sentence_n_words_from_documents(documents).apply(lambda o: sum(map(lambda l : l <= self._n_words_threshold, o)) / len(o)))

    return np.mean(sample_level_accuracies)

class SplittingEvaluationMetricsCalculation:
  def __init__ (self, target_maximum_n_words: int):
    self._bertscore_metric = evaluate.load("bertscore", module_type="metric")

    self._sacre_bleu_metric = evaluate.load('sacrebleu', module_type="metric")

    self._word_based_accuracy_metric = SplittingWordBasedAccuracy(n_words_threshold=target_maximum_n_words)

  def _bert_score_f1(self, predictions: pd.Series, references: pd.Series, batch_size: int) -> float:
      batch_scores = []
      
      for i in tqdm(range(0, len(predictions), batch_size)):
          predictions_batch = predictions[i : i + batch_size]

          references_batch = references[i : i + batch_size]

          batch_f1 = self._bertscore_metric.compute(predictions=predictions_batch, references=references_batch, lang='en', batch_size=batch_size) ['f1']

          batch_scores.extend(batch_f1)

      return np.mean(batch_scores)

  def compute(self, predictions: pd.Series, sources: pd.Series, targets: pd.Series) -> pd.Series:
    predictions, sources, targets = predictions.values, sources.values, targets.values

    bert_score_f1_with_source = self._bert_score_f1(predictions=predictions, references=sources, batch_size=20)
    bert_score_f1_with_target = self._bert_score_f1(predictions=predictions, references=targets, batch_size=20)

    sacre_bleu_score_with_source = self._sacre_bleu_metric.compute(predictions=predictions, references=sources) ['score']
    sacre_bleu_score_with_target = self._sacre_bleu_metric.compute(predictions=predictions, references=targets) ['score']

    accuracy_percentage = 100.0 * self._word_based_accuracy_metric.compute(predictions)

    return pd.Series([
        sacre_bleu_score_with_source,
        sacre_bleu_score_with_target,

        bert_score_f1_with_source,
        bert_score_f1_with_target,

        accuracy_percentage
    ], index=[ 'SacreBLEU (s)',
               'SacreBLEU (t)',
               'BERTScore F1 (s)',
               'BERTScore F1 (t)',
               'Accuracy (%)' ])

# Connect Google Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

# Create paths

In [ ]:
MODELING_DATA_DIRECTORY_PATH = Path('/content/drive/My Drive/Pfizer - Long Sentence Resolution/Datasets/modeling')

MODELS_DIRECTORY_PATH = Path('/content/drive/My Drive/Pfizer - Long Sentence Resolution/Models')

# Configuration parameters

In [ ]:
test_set_generated_predictions_source_dropdown = widgets.Dropdown(options=list(filter(lambda o: o.startswith('splitting_model'), os.listdir(MODELS_DIRECTORY_PATH))))
test_set_ground_truth_source_dropdown = widgets.Dropdown(options=list(filter(lambda o: o.startswith('splitting_dataset'), os.listdir(MODELING_DATA_DIRECTORY_PATH))))
target_maximum_n_words_bounded_int_text = widgets.BoundedIntText(value=16, min=1, max=20, step=1)

display(widgets.HBox([widgets.Label(value="Source of test set generated predictions:"), test_set_generated_predictions_source_dropdown]))
display(widgets.HBox([widgets.Label(value="Source of test set ground-truth:"), test_set_ground_truth_source_dropdown]))
display(widgets.HBox([widgets.Label(value="Target maximum #words:"), target_maximum_n_words_bounded_int_text]))

# Load test set generated predictions & ground-truth

In [ ]:
with open(MODELS_DIRECTORY_PATH / test_set_generated_predictions_source_dropdown.value / 'generated_predictions.txt', 'r') as f: predictions_df = pd.DataFrame(f.read().splitlines(), columns=['prediction'])

predictions_df

In [ ]:
test_set_df = pd.read_csv(MODELING_DATA_DIRECTORY_PATH / test_set_ground_truth_source_dropdown.value / 'test.csv', usecols=['dataset', 'source', 'target'])

test_set_df

In [ ]:
assert(len(test_set_df) == len(predictions_df))

In [ ]:
test_set_df['prediction'] = predictions_df['prediction']

test_set_df

# Calculate metrics per dataset

In [ ]:
%%time

evaluation_metrics_calculation = SplittingEvaluationMetricsCalculation(target_maximum_n_words=target_maximum_n_words_bounded_int_text.value)

test_set_df.groupby('dataset').apply(lambda o: evaluation_metrics_calculation.compute(o['prediction'], o['source'], o['target']))